<a href="https://colab.research.google.com/github/Yash8743/qml/blob/main/Task_10(QML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TASK 10: Implement the QAOA algorithm**

**Aim:**
To implement the Quantum Approximate Optimization Algorithm (QAOA) using Qiskit
and PyTorch to solve the Max-Cut problem, a classical NP-hard problem

**Mathematical Model of the QAOA Algorithm**

1.1 Max-Cut Problem Formulation
Given a graph 𝐺 = (𝑉, 𝐸) with weighted adjacency matrix 𝑤௜௝ , the Max-Cut objective is
𝐶(𝑥) = ∑(௜,௝)ఢா 𝑤௜௝ . ൫𝑥௜ 𝑥௝൯
where 𝑥௜{0,1} indicates the partition of node 𝑖.
The goal is ௠௔௫
௫{଴,ଵ}೙ 𝐶(𝑥)

1.2 QAOA Ansatz
The QAOA prepares a quantum state parameterized by angles 𝛾⃗, 𝛽⃗.
ห൫𝛾⃗, 𝛽⃗൯〉 = ෑ𝑒
ି௜ఉ೗ ∑೔ ௑೔
௣
௟ୀଵ
𝑒
ି௜ఊ೗ு಴ห+〉௡
 𝐻஼ = ∑(௜,௝)ఢா 𝑤௜௝𝑍௜𝑍௝ (cost Hamiltonian)
 𝐻஻ = ∑௜ 𝑋௜ (mixer Hamiltonian)

1.3 Expectation Value
The objective is to maximize𝐹൫𝛾⃗, 𝛽⃗൯ = 〈൫𝛾⃗, 𝛽⃗൯ |Hେ
|൫𝛾⃗, 𝛽⃗൯〉, the optimization is performed
using a classical optimizer (Adam with finite-difference gradients).

**Algorithm - QAOA Algorithm**

1. Graph Construction
 Define adjacency matrix W.
 Build a NetworkX graph for visualization.

2. Classical Baseline
 Use brute-force enumeration to compute the optimal Max-Cut value (ground truth).

3. QAOA Circuit Construction
 Initialize qubits in |+〉 .
 Apply alternating cost and mixer unitaries for depth 𝑝.
 Use controlled-𝑍 rotation gates to implement 𝑍௜𝑍௝
 interactions.

4. Expectation Calculation
 Simulate circuit using Qiskit Aer statevector simulator.
 Compute expected cut value from measurement probabilities.

5. Hybrid Optimization
 Parameters ൫𝛾⃗, 𝛽⃗൯ initialized randomly.
 Compute finite-difference gradients of expectation.
 Update parameters using PyTorch Adam optimizer.

6. Circuit Visualization
 Draw initial and optimized QAOA circuits using qiskit.visualization.

**Program:**

In [ ]:
# =============================
# Install dependencies (Colab)
# =============================
!pip install qiskit qiskit-aer qiskit-optimization torch networkx numpy matplotlib pylatexenc --quiet

# =============================
# Imports
# =============================
import os
import numpy as np
import networkx as nx
import torch
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
import matplotlib

# Headless environments fix
matplotlib.use(os.environ.get("MPLBACKEND", "Agg"))
import matplotlib.pyplot as plt


# =============================
# Problem definition
# =============================
def make_graph():
    # 4-node weighted graph (like Qiskit MaxCut tutorial)
    w = np.array([
        [0.0, 1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0, 1.0],
        [1.0, 1.0, 0.0, 1.0],
        [0.0, 1.0, 1.0, 0.0]
    ])
    G = nx.from_numpy_array(w)
    return G, w


def objective_value(x, w):
    """Compute cut value for a bitstring x"""
    X = np.outer(x, (1 - x))
    return np.sum(w * X)  # use full weights


def brute_force_maxcut(w):
    """Brute-force search (for small graphs)"""
    n = w.shape[0]
    best = -1
    best_x = None
    for i in range(2 ** n):
        x = np.array(list(map(int, np.binary_repr(i, width=n))))
        val = objective_value(x, w)
        if val > best:
            best = val
            best_x = x
    return best_x, best


# =============================
# QAOA Circuit
# =============================
def qaoa_circuit(n_qubits, edges, gammas, betas):
    """Build a QAOA circuit manually"""
    p = len(gammas)
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))  # initial state

    for layer in range(p):
        gamma = float(gammas[layer])
        for (i, j, w) in edges:
            if w == 0:
                continue
            theta = 2.0 * gamma * w
            qc.cx(i, j)
            qc.rz(theta, j)
            qc.cx(i, j)
        beta = float(betas[layer])
        for q in range(n_qubits):
            qc.rx(2.0 * beta, q)

    return qc


# =============================
# Expectation
# =============================
def expectation_from_statevector(statevector, w):
    """Compute expected cut value from statevector"""
    n = w.shape[0]
    probs = Statevector(statevector).probabilities_dict()
    exp_val = 0.0
    for bitstr, p in probs.items():
        bits = np.array([int(b) for b in bitstr[::-1]])
        exp_val += objective_value(bits, w) * p
    return exp_val


# =============================
# Optimization Loop (PyTorch + finite diff)
# =============================
def run_qaoa_with_pytorch(w, p=1, init_std=0.5, maxiter=100,
                          lr=0.1, finite_diff_eps=1e-3,
                          backend_name="aer_simulator_statevector"):
    n = w.shape[0]
    edges = [(i, j, w[i, j]) for i in range(n) for j in range(i) if w[i, j] != 0]

    params = torch.randn(2 * p, dtype=torch.double) * init_std

    backend = Aer.get_backend(backend_name)
    best = {"val": -np.inf, "params": None, "bitstring": None}

    for it in range(maxiter):
        base = params.detach().numpy()
        gammas, betas = base[:p], base[p:]

        qc = qaoa_circuit(n, edges, gammas, betas)
        qc.save_statevector()
        res = backend.run(qc).result()
        sv = res.get_statevector(qc)

        exp_val = expectation_from_statevector(sv, w)
        loss = -exp_val

        # record best
        if exp_val > best["val"]:
            probs = Statevector(sv).probabilities_dict()
            most = max(probs.items(), key=lambda kv: kv[1])[0]
            bits = np.array([int(b) for b in most[::-1]])
            best.update({"val": exp_val,
                         "params": params.clone(),
                         "bitstring": bits})

        # finite-difference gradient
        grads = np.zeros_like(base)
        eps = finite_diff_eps
        for k in range(len(base)):
            plus, minus = base.copy(), base.copy()
            plus[k] += eps
            minus[k] -= eps
            g_plus = _qaoa_expectation_with_params(plus, n, edges, backend, w, p)
            g_minus = _qaoa_expectation_with_params(minus, n, edges, backend, w, p)
            grads[k] = -((g_plus - g_minus) / (2 * eps))

        params = params - lr * torch.from_numpy(grads).to(dtype=torch.double)

        if it % 10 == 0 or it == maxiter - 1:
            print(f"Iter {it:03d}: expected cut = {exp_val:.6f}, loss = {loss:.6f}")

    return best


def _qaoa_expectation_with_params(flat_params, n, edges, backend, w, p):
    gammas, betas = flat_params[:p], flat_params[p:]
    qc = qaoa_circuit(n, edges, gammas, betas)
    qc.save_statevector()
    res = backend.run(qc).result()
    sv = res.get_statevector(qc)
    return expectation_from_statevector(sv, w)


# =============================
# Circuit Display Helpers
# =============================
def show_circuit(qc: QuantumCircuit, filename: str = None, style: str = "mpl"):
    print("\n--- Quantum Circuit ---")
    try:
        print(qc.draw(output="text"))
    except Exception as e:
        print("Text draw failed:", e)

    if style == "mpl":
        try:
            fig = qc.draw(output="mpl", interactive=False)
            fig.tight_layout()
            fname = filename if filename else "qaoa_circuit.png"
            fig.savefig(fname, dpi=200, bbox_inches="tight")
            print(f"[Saved circuit figure to {fname}]")
            plt.close(fig)
        except Exception as e:
            print("Matplotlib drawing failed:", e)


def demo_display_initial_circuit(w, p=1, filename="qaoa_initial_circuit.png"):
    n = w.shape[0]
    gammas, betas = np.random.randn(p) * 0.8, np.random.randn(p) * 0.8
    edges = [(i, j, w[i, j]) for i in range(n) for j in range(i) if w[i, j] != 0]
    qc = qaoa_circuit(n, edges, gammas, betas)
    show_circuit(qc, filename=filename, style="mpl")


def demo_display_best_circuit(w, best_params, p=1, filename="qaoa_best_circuit.png"):
    n = w.shape[0]
    flat = best_params.detach().cpu().numpy() if isinstance(best_params, torch.Tensor) else np.array(best_params)
    gammas, betas = flat[:p], flat[p:]
    edges = [(i, j, w[i, j]) for i in range(n) for j in range(i) if w[i, j] != 0]
    qc = qaoa_circuit(n, edges, gammas, betas)
    show_circuit(qc, filename=filename, style="mpl")


# =============================
# Run Demo
# =============================
if __name__ == "__main__":
    G, w = make_graph()
    print("Graph edges:", list(G.edges()))
    bf_x, bf_val = brute_force_maxcut(w)
    print("Brute-force best:", bf_x, "value:", bf_val)

    demo_display_initial_circuit(w, p=1, filename="qaoa_initial_circuit.png")

    best = run_qaoa_with_pytorch(w, p=1, init_std=0.8, maxiter=50, lr=0.2, finite_diff_eps=1e-3)
    print("QAOA best expected value:", best["val"])




    print("Most-likely bitstring found:", best["bitstring"])
    exact_val = objective_value(best["bitstring"], w)
    print("Exact value of that bitstring:", exact_val)

    if best["params"] is not None:
        demo_display_best_circuit(w, best["params"], p=1, filename="qaoa_best_circuit.png")
    else:
        print("No best params found to display.")


Graph edges: [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)]
Brute-force best: [0 1 1 0] value: 4.0

--- Quantum Circuit ---
     ┌───┐┌───┐┌─────────────┐┌───┐┌───┐┌─────────────┐┌───┐┌─────────────┐»
q_0: ┤ H ├┤ X ├┤ Rz(-3.1843) ├┤ X ├┤ X ├┤ Rz(-3.1843) ├┤ X ├┤ Rx(0.68826) ├»
     ├───┤└─┬─┘└─────────────┘└─┬─┘└─┬─┘└─────────────┘└─┬─┘└────┬───┬────┘»
q_1: ┤ H ├──■───────────────────■────┼───────────────────┼───────┤ X ├─────»
     ├───┤                           │                   │       └─┬─┘     »
q_2: ┤ H ├───────────────────────────■───────────────────■─────────■───────»
     ├───┤                                                                 »
q_3: ┤ H ├─────────────────────────────────────────────────────────────────»
     └───┘                                                                 »
«                                                                 »
«q_0: ────────────────────────────────────────────────────────────»
«     ┌─────────────┐┌───┐┌───┐┌─────────────┐┌───┐┌─

**Result:**
The QAOA implementation successfully demonstrates a hybrid quantum-classical optimization
approach to solving the Max-Cut problem.